In [2]:
#Data processing
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nptdms import TdmsFile #Process ramping file

#For building ML models
import keras
import keras.models as models
from keras.layers.core import Dense
from keras import regularizers

In [3]:
def load_channel_and_time(dir_path, channel):
    data_frame = pd.DataFrame(data = {channel: np.load(dir_path + channel + ".npy"),
                                     "time": np.load(dir_path + "time.npy")})
    return data_frame
    
def generate_data_0(dir_path, channel, time_range, window = 1000, step = 10):
    #Load the data
    data = load_channel_and_time(dir_path, channel)
    
    #Select the part
    start = time_range[0]
    end = time_range[1]
    data = data[(data["time"] > start) & (data["time"] < end)]
    
    #Calculate the statistics
    #data["Mean"] = data.loc[:, channel].abs().rolling(window=window).mean()
    data["SD"] = data.loc[:, channel].rolling(window=window).std()
    #data["Kurtosis"] = data.loc[:, channel].rolling(window=window).kurt()
    #data["Skew"] = data.loc[:, channel].rolling(window=window).skew()
    
    select_list = ["SD"]
    
    assert data[select_list].to_numpy()[window-1::step].shape[0] == data['time'].to_numpy()[window-1::step].shape[0]
    
    %reset -f in
    
    return data[select_list].to_numpy()[window-1::step], data['time'].to_numpy()[window-1::step]

def generate_data_no_time(dir_path, channel, time_range, window = 1000, step = 10):
    #Load the data
    data = load_channel_and_time(dir_path, channel)
    
    #Select the part
    start = time_range[0]
    end = time_range[1]
    data = data[(data["time"] > start) & (data["time"] < end)]
    
    #Calculate the statistics
    #data["Mean"] = data.loc[:, channel].abs().rolling(window=window).mean()
    data["SD"] = data.loc[:, channel].rolling(window=window).std()
    #data["Kurtosis"] = data.loc[:, channel].rolling(window=window).kurt()
    #data["Skew"] = data.loc[:, channel].rolling(window=window).skew()
    
    select_list = ["SD"]
    
    %reset -f in
    
    return data[select_list].to_numpy()[window-1::step]

def generate_data_all_sensors(dir_path, time_range, window = 1000, step = 10):
    
    ai0, time = generate_data(dir_path, "ai0", time_range = time_range, window = window, step = step)
    ai1 = generate_data_no_time(dir_path, "ai1", time_range = time_range, window = window, step = step)
    ai2 = generate_data_no_time(dir_path, "ai2", time_range = time_range, window = window, step = step)
    ai3 = generate_data_no_time(dir_path, "ai3", time_range = time_range, window = window, step = step)
    ai4 = generate_data_no_time(dir_path, "ai4", time_range = time_range, window = window, step = step)
    
    #Multiply them all together
    product_var = ai0*ai1*ai2*ai3*ai4
    
    
    all_channels = np.concatenate((ai0,ai1,ai2,ai3,ai4,product_var), axis = 1)
    
    %reset -f in
    
    return all_channels, time

def plot_moving_mean(dir_path, channel, time_range, window = 1000, step = 10):
    #Load the data
    data = load_channel_and_time(dir_path, channel)
    
    #Select the part
    start = time_range[0]
    end = time_range[1]
    data = data[(data["time"] > start) & (data["time"] < end)]
    
    #Calculate the mean
    data["Mean"] = data.loc[:, channel].abs().rolling(window=window).mean()
    
    #Plot
    plt.figure(figsize=(20,2))
    plt.plot(data['time'].to_numpy()[window-1::step], data["Mean"].to_numpy()[window-1::step], label = "Mean of abs(data)")
    plt.legend(loc = "upper right")
    plt.xlabel("Time (s)")
    plt.title("Sensor {}'s moving mean".format(channel))
    
    %reset -f in
    
def load_sensor(dir_path, sensor, time_range = None):
    
    data = pd.DataFrame(data = {sensor: np.load(dir_path + sensor + ".npy"),
                                "time": np.load(dir_path + "time.npy")})
    
    start = min(data["time"])
    end = max(data["time"])
    
    if time_range:
        start = time_range[0]
        end = time_range[1]
    
    
    data = data[(data["time"] > start) & (data["time"] < end)]
   
    %reset -f in
    
    return data
    
def plot_product_mean(dir_path, time_range, window = 1000, step = 10):
    #Load the data
    ai0 = load_sensor(dir_path, "ai0", time_range = time_range)
    ai1 = load_sensor(dir_path, "ai1", time_range = time_range)["ai1"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai2 = load_sensor(dir_path, "ai2", time_range = time_range)["ai2"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai3 = load_sensor(dir_path, "ai3", time_range = time_range)["ai3"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai4 = load_sensor(dir_path, "ai4", time_range = time_range)["ai4"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    
    time_axis = ai0['time'].to_numpy()[window-1::step]
    
    ai0 = ai0["ai0"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    
    product = ai0*ai1*ai2*ai3*ai4
    
    #Plot
    plt.figure(figsize=(20,2))
    plt.plot(time_axis, product)
    plt.xlabel("Time (s)")
    plt.title("Product of moving means")
    
def plot_SD(dir_path, channel, time_range, window = 2000, step = 10):
    
    #Load the data
    data = load_channel_and_time(dir_path, channel)
    
    #Select the part
    start = time_range[0]
    end = time_range[1]
    data = data[(data["time"] > start) & (data["time"] < end)]
    
    #Calculate the mean
    data["SD"] = data.loc[:, channel].abs().rolling(window=window).std()
    
    #Plot
    plt.figure(figsize=(20,2))
    plt.plot(data['time'].to_numpy()[window-1::step], data["SD"].to_numpy()[window-1::step], label = "Standard deviation", color = "red")
    plt.legend(loc = "upper right")
    plt.xlabel("Time (s)")
    plt.title("Sensor {}'s moving standard deviation".format(channel))
    
    %reset -f in


def plot_product_SD(dir_path, time_range, window = 1000, step = 10):
    #Load the data
    ai0 = load_sensor(dir_path, "ai0", time_range = time_range)
    ai1 = load_sensor(dir_path, "ai1", time_range = time_range)["ai1"].rolling(window=window).std().to_numpy()[window-1::step]
    ai2 = load_sensor(dir_path, "ai2", time_range = time_range)["ai2"].rolling(window=window).std().to_numpy()[window-1::step]
    ai3 = load_sensor(dir_path, "ai3", time_range = time_range)["ai3"].rolling(window=window).std().to_numpy()[window-1::step]
    ai4 = load_sensor(dir_path, "ai4", time_range = time_range)["ai4"].rolling(window=window).std().to_numpy()[window-1::step]
    
    time_axis = ai0['time'].to_numpy()[window-1::step]
    
    ai0 = ai0["ai0"].rolling(window=window).std().to_numpy()[window-1::step]
    
    product = ai0*ai1*ai2*ai3*ai4
    
    #Plot
    plt.figure(figsize=(20,2))
    plt.plot(time_axis, product, color = "red")
    plt.xlabel("Time (s)")
    plt.title("Product of moving standard deviations")
    
def generate_mean_data(dir_path, time_range, window = 2000, step = 10):
    #Load the data
    ai0 = load_sensor(dir_path, "ai0", time_range = time_range)["ai0"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai1 = load_sensor(dir_path, "ai1", time_range = time_range)["ai1"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai2 = load_sensor(dir_path, "ai2", time_range = time_range)["ai2"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai3 = load_sensor(dir_path, "ai3", time_range = time_range)["ai3"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    ai4 = load_sensor(dir_path, "ai4", time_range = time_range)["ai4"].abs().rolling(window=window).mean().to_numpy()[window-1::step]
    
    #Calculate the product
    product = ai0*ai1*ai2*ai3*ai4
    
    #Stack them together
    all_mean = np.vstack((ai0,ai1,ai2,ai3,ai4, product)).transpose()
    
    %reset -f in
    
    return all_mean

def generate_sd_data(dir_path, time_range, window = 2000, step = 10):
    #Load the data
    ai0 = load_sensor(dir_path, "ai0", time_range = time_range)["ai0"].rolling(window=window).std().to_numpy()[window-1::step]
    ai1 = load_sensor(dir_path, "ai1", time_range = time_range)["ai1"].rolling(window=window).std().to_numpy()[window-1::step]
    ai2 = load_sensor(dir_path, "ai2", time_range = time_range)["ai2"].rolling(window=window).std().to_numpy()[window-1::step]
    ai3 = load_sensor(dir_path, "ai3", time_range = time_range)["ai3"].rolling(window=window).std().to_numpy()[window-1::step]
    ai4 = load_sensor(dir_path, "ai4", time_range = time_range)["ai4"].rolling(window=window).std().to_numpy()[window-1::step]
    
    #Calculate the product
    product = ai0*ai1*ai2*ai3*ai4
    
    #Stack them together
    all_sd = np.vstack((ai0,ai1,ai2,ai3,ai4, product)).transpose()
    
    %reset -f in
    
    return all_sd

def load_time_label(dir_path, time_range, window = 2000, step = 10):
    
    time_label =  np.load(dir_path + "time.npy")
    
    start = min(time_label)
    end = max(time_label)
    
    if time_range:
        start = time_range[0]
        end = time_range[1]
    
    
    time_label = time_label[(time_label > start) & (time_label < end)][window-1::step]
    
    %reset -f in
    
    return time_label
    

def generate_data(dir_path, time_range, window = 2000, step = 10):
    
    moving_mean = generate_mean_data(dir_path, time_range, window = 2000, step = 10)
    moving_sd = generate_sd_data(dir_path, time_range, window = 2000, step = 10)
    time_label = load_time_label(dir_path, time_range, window = 2000, step = 10)
    
    all_data = np.concatenate((moving_mean, moving_sd), axis = 1)
    
    %reset -f in
    
    return all_data, time_label

In [7]:
X_train, train_time = generate_data("./data/Ramp21/", time_range = (-450, -420), window = 2000, step = 10)

Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history
Flushing input history


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)

#X_test = scaler.transform(X_test)

In [10]:
#================BUILD THE MODEL====================
print("Using Keras version: ", keras.__version__)

# Simple model
model = models.Sequential()

model.add(Dense(6, activation = 'elu', kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.0),
                input_dim=X_train.shape[1]))

model.add(Dense(3,activation='elu',
                kernel_initializer='glorot_uniform'))

model.add(Dense(6,activation='elu',
                kernel_initializer='glorot_uniform'))

model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

model.compile(loss='mse',optimizer='adam')

model.summary()

Using Keras version:  2.3.1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 24        
_________________________________________________________________
dense_4 (Dense)              (None, 12)                84        
Total params: 207
Trainable params: 207
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train model for 100 epochs, batch size of 10: 
NUM_EPOCHS=20
BATCH_SIZE=1028

history=model.fit(X_train, X_train,
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.1,
                  verbose = 1)

Train on 269820 samples, validate on 29980 samples
Epoch 1/20
269820/269820 [==============================] - 3s 12us/step - loss: 0.0022 - val_loss: 0.0025
Epoch 2/20
269820/269820 [==============================] - 2s 7us/step - loss: 6.3256e-04 - val_loss: 0.0016
Epoch 3/20
269820/269820 [==============================] - 2s 6us/step - loss: 3.6246e-04 - val_loss: 0.0014
Epoch 4/20
269820/269820 [==============================] - 2s 6us/step - loss: 2.5459e-04 - val_loss: 0.0014
Epoch 5/20
269820/269820 [==============================] - 2s 7us/step - loss: 2.2022e-04 - val_loss: 0.0013
Epoch 6/20
269820/269820 [==============================] - 2s 6us/step - loss: 2.0654e-04 - val_loss: 0.0012
Epoch 7/20
269820/269820 [==============================] - 2s 6us/step - loss: 1.9849e-04 - val_loss: 0.0011
Epoch 8/20
269820/269820 [==============================] - 2s 6us/step - loss: 1.9211e-04 - val_loss: 0.0011
Epoch 9/20
269820/269820 [==============================] - 2s 7us/step 

In [14]:
model.save("models/Ramp21-12in.h5")

In [15]:
!pwd

/uscms_data/d3/dhoang/quench-detector
